In [79]:
import numpy as np
import time
from copy import deepcopy

# Import Qiskit classes
import qiskit
import qiskit.quantum_info as qi
from qiskit import QuantumRegister, QuantumCircuit, ClassicalRegister, Aer
from qiskit.providers.aer import noise
from qiskit.compiler import assemble

## For using custom unitaries as gates
from qiskit import transpile


# Gates
---


Classical Checkers has the option of two moves only, the diagonal jump to the nearest diagonal and the capture jump over another piece in the diagonal direction. Our game introduces two new moves in order to demonstrate the effects of superposition and entanglement to the game. These two new moves include the split jump, the puts a piece into superposition of two nearest diagonals and the merge jump that allows a player to return a piece back from super position back into just one box. Hence, we have constructed four unitaries at large in order to implement these four moves:

1. Jump Unitary (Diagonal Jump).
2. Capture (Capture Jump).
3. Uslide (Quantum Split Jump).
4. Umerge (Quantum Merge Jump).


#### Custom Unitaries to implement Simple jump, Capture and Quantum Split Jump & Quantum Merge Jump.


- Jump Unitary


    ![jump](jump.png)


*Advantage of using this instead of a simple  swap?

$\newline$
- Capture


     ![split](Capture.png)






In [80]:
Ujump = [[1, 0, 0, 0],
         [0, 0, 1.j, 0],
         [0, 1.j, 0, 0],
         [0, 0, 0, 1]]
         


Capture=[[1,0,0,0,0,0,0,0],
     [0,0,0,0,1,0,0,0],
     [0,0,0,0,0,1,0,0],
     [0,0,0,0,0,0,1,0],
     [0,1,0,0,0,0,0,0],
     [0,0,1,0,0,0,0,0],
     [0,0,0,1,0,0,0,0],
     [0,0,0,0,0,0,0,1]]


## Checking if the Capture is unitary 
Capturedag=np.matrix(Capture).getH()
print(np.dot(Capture,Capturedag))

[[1 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0]
 [0 0 0 1 0 0 0 0]
 [0 0 0 0 1 0 0 0]
 [0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 1]]


## Split jump with path consideration

Before implementiong the complete Uslide, we constructed two other unitaries for considerations of a **single path or both paths being blocked**. In case where both paths are open, we use the split jump, when both are blocked the piece stays in position and when only one is open the piece moves in the open direction without going into superposition.

$\newline$
- Split

 ![jump](split.png)

- iSwap

 ![jump](iswap.png)


We apply a Split slide Unitary using the above mentioned gates:

 ![jump](splitslide.png)
 

In [81]:
Uslide = [
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 1j, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 1.j/np.sqrt(2), 1/np.sqrt(2), 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, -1/np.sqrt(2), 1.j/np.sqrt(2), 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 1.j/np.sqrt(2), -1/np.sqrt(2), 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 1.j/np.sqrt(2), -1/np.sqrt(2), 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 1j, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1j, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1j, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1j, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1j, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1j, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1j, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1j, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1j, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
]


Umerge= np.matrix(Ujump).getH()

# Move Functions
---
Following functions are implemented for each move. The custom unitaries are used to update the occupany bits and some conventional quantum gates are used to update the color qubits of each box according to the action of the unitary.


In [82]:
## color qubit updates

## Updates colors on slide split.
def updateCColSplit(to1,to2,t1,t2,s,ancila, qc):
     qc.cnot(s,ancila)
     qc.cnot(ancila,s)
     qc.swap(ancila,s)
     qc.cswap(to1,ancila,t1)
     qc.cswap(to2,ancila,t2)


## Update colors on Capture
def updateColcapture(t,p,s,ancila, qc):
     qc.swap(s,t)
     qc.cnot(p, ancila)
     qc.cnot(ancila, p)


## Occupancy Updates

def split_jump_updated(t1,t2,s,ancila, qc):
     # qc.unitary(Usplit,[s, t1, t2])
     qc.cx(t1, 37)
     qc.cx(t2, 38)

     qc.unitary(Uslide, [s,t1,t2,37,38])
     updateCColSplit(t1, t2, t1+18, t2+18, s, ancila, qc)


def jump_updated(t1, s, qc):
     qc.swap(s, t1)
     qc.swap(s+18, t1+18)


def capture(t1, s, p, ancila, qc):
     print(f"Capture: t1={t1} s={s} p={p}")
     qc.unitary(Cap, [s, t1, p])
     updateColcapture(t1+18, p+18, s+18, ancila, qc)

# State of the Board:
---

We have chosen the dimensions of 6x6 board which gives us a total of 13 boxes (all black) which can be occupied at any given time. The state of the board consists of (13 * 2) qubits, meaning each box in the grid is represented in 2 qubits. The first qubit represents the occupancy of the box, 1 in case of being occupied and 0 if not occupied. The second qubit will represent the color of the piece in it, 0 if white and 1 if black.

In our circuit we have chosen to declare a total of 29 qubits with first 13 qubits to carry the occupancy states of each box, next thirteen qubits will carry the color states of each corresponding occupancy. The last three qubits are ancillary and are useful for the operations of capture, quantum split and merge as well.

In [83]:
def get_new_circuit(counts):
    qr = QuantumRegister(39, "box")
    cr = ClassicalRegister(36)
    qc = QuantumCircuit(qCapture, cCapture)
    
    count = 0

    for i in list(counts.keys())[0][0:36]:
        if i ==  '1':
            qc.x(count)
        
        count +=1
    
    return qc


def print_state_board2(counts, size):
    state = []

    counts_temp = {}

    for i in counts:
        i_ = i.replace(" ", "")
        counts_temp[i_] = counts[i]

    counts = counts_temp
    
    count = 0
    count_alt = False
    row = ""

    # print(counts),

    for i in range(0, 18):

        # i = i//2,

        box = " "
        
        for k in counts:
            if k[i] == '1':
                if k[i+18] == '1':
                    if box == " ":
                        box = "B"
                    else:
                        box += "B"
                else:
                    if box == " ":
                        box = "W"
                    else:
                        box += "W"
        
        # print(type(row), type(box))
        row += box + '\t\t'

        count+=1

        if count == 3:
            count = 0
            if count_alt:
                state.append("\t\t" + row[:-2])
            else:
                state.append(row[:-2])

            count_alt = not count_alt
            # print(row[0])
            row = ""


    state = state[::-1]

    output = ""
    
    for i in state[:-1]:
        # print(i)
        # print("-----------------" * (size//2))
        output += i + "\n"
        output += ("     " * (size//2)) + ("------------------" * (size//2)) + "\n"
    
    # print(state[-1])
    output += state[-1] + "\n"

    # print(output)
    
    return output


def update_classical_state(counts, s, t1, t2=None):

    print(s,t1,t2)

    for i in list(counts.keys())[0:1]:
        temp = list(i)

        temp[t1] = temp[s]
        temp[t1+18] = temp[s+18]

        if t2 != None:
            temp[t2] = temp[s]
            temp[t2+18] = temp[s+18]
        
        temp[s] = '0'
        temp[s+18] = '0'
        
        
        print(temp)
        
        temp = "".join(temp)
        
        counts[temp] = counts[i]
        del counts[i]
    
    # print_state_board2(counts, 5)
    print("\n\n\n")


from tkinter import *

root = Tk()  
root.geometry("400x490")  
root.title("test")

# canvas = Canvas(root)

V = StringVar()
label = Label(root, textvariable=V, bg = "white", bd=100, fg = "black")
label.pack()
# label.place(x=-160, y=0)


l1 = Label(root, text="source box: ")
l1.place(x=80, y=360)
s = Entry(root)
s.place(x=150,y=360)
# s.pack()

l2 = Label(root, text="t1 box: ")
l2.place(x=10, y=400)
t1 = Entry(root)
t1.place(x=50, y=400)
# t1.pack()

l3 = Label(root, text="t2 box: ")
l3.place(x=210, y=400)
t2 = Entry(root)
t2.place(x=250, y=400)
# t2.pack()


def gui_jump():
    jump_updated(int(t1.get()), int(s.get()), bell)
    # print(qi.Statevector.from_instruction(bell.reverse_bits()).to_dict())
    update_classical_state(counts, int(s.get()), int(t1.get()))

    V.set(print_state_board2(counts, 6))
    
def gui_sjump():
    split_jump_updated(int(t1.get()), int(t2.get()), int(s.get()), 36, bell)
    # print(qi.Statevector.from_instruction(bell.reverse_bits()).to_dict())
    update_classical_state(counts, int(s.get()), int(t1.get()), int(t2.get()))

    V.set(print_state_board2(counts, 6))

def gui_Captureture():
    if int(s.get()) < int(t1.get()):
        if (int(t1.get()) - int(s.get())) < 7:
            if up_diag[int(s.get())][0] != -1:
                path_qb = up_diag[int(s.get())][0]
                Captureture(int(t1.get()), int(s.get()), path_qb, 36, bell)
        else:
            if up_diag[int(s.get())][1] != -1:
                path_qb = up_diag[int(s.get())][1]
                Captureture(int(t1.get()), int(s.get()), path_qb, 36, bell)
                
        # Captureture(int(t1.get()), int(s.get()), int(s.get())+3, 36, bell)
    else:
        if (int(s.get()) - int(t1.get())) < 7:
            if down_diag[int(s.get())][1] != -1:
                path_qb = down_diag[int(s.get())][1]
                Captureture(int(t1.get()), int(s.get()), path_qb, 36, bell)
        else:
            if down_diag[int(s.get())][0] != -1:
                path_qb = down_diag[int(s.get())][0]
                Captureture(int(t1.get()), int(s.get()), path_qb, 36, bell)
        # Captureture(int(t1.get()), int(s.get()), int(s.get())-3, 36, bell)

def gui_measure():
    global bell
    bell.measure(list(range(36)), list(range(35,-1,-1)))
    # bell.measure([3,4,5,6,7], [4,3,2,1,0])
    job = execute(bell,Aer.get_backend('qasm_simulator'),shots=1)
    global counts
    counts = job.result().get_counts(bell)
    bell = get_new_circuit(counts)
    print(counts)
    V.set(print_state_board2(counts, 6))


b1 = Button(root, text ="Jump", command=gui_jump)
b1.place(x=120, y=425)

b2 = Button(root, text ="Split Jump", command=gui_sjump)
b2.place(x=170, y=425)
# b.pack()

b3 = Button(root, text ="Captureture", command=gui_Captureture)
b3.place(x=250, y=425)

b4 = Button(root, text ="Measure", command=gui_measure)
b4.place(x=160, y=460)


V.set(print_state_board2(counts, 6))


root.mainloop()


## Moves
- Jump Unitary


    ![jump](jump.png)



$\newline$
- Split


     ![split](split.png)




$\newline$
- Merge


    ![merge](merge.png)

In [84]:
def jump(src, target, qc):
    for i in range(2):
        qc.unitary(Ujump, [src[i], target[i]])

def split(src, target1, target2, qc):
    # for i in range(2):
    #     qc.unitary(Usplit, [src[i], target1[i], target2[i]])
    # qc.unitary(Usplit, [src[1], target1[1], target2[1]])
    qc.unitary(Usplit, [src[0], target1[0], target2[0]])

def merge(src1, src2, target, qc):
    qc.unitary(Umerge, [src1[0], src2[0], target[0]])
    qc.unitary(Umerge, [src1[1], src2[1], target[1]])




In [85]:
def get_diag_white(box):
    return (pos_dic[box][0]+1, pos_dic[box][1]+1), (pos_dic[box][0]-1, pos_dic[box][1]+1)  

def get_diag_black(box):
    return (pos_dic[box][0]+1, pos_dic[box][1]-1), (pos_dic[box][0]-1, pos_dic[box][1]-1)  

 
src = 'box_(1,1)'
diag1, diag2 = get_diag_white('box_(1,1)')

# print(diag1, diag2)
# print(diag1)
# print(box_to_qubit.index(diag1))
src_bits = (box_to_qubit.index((1,1)), box_to_qubit.index((1,1))+1)
target1_bits = (box_to_qubit.index(diag1), box_to_qubit.index(diag1) + 1)
target2_bits = (box_to_qubit.index(diag2), box_to_qubit.index(diag2) + 1)

# jump(src_bits, target1_bits, q_board)
# split(src_bits, target1_bits, target2_bits, q_board)


src = 'box_(4,4)'
diag1, diag2 = get_diag_white('box_(4,4)')

src_bits = (box_to_qubit.index((4,4)), box_to_qubit.index((4,4))+1)
target1_bits = (box_to_qubit.index(diag1), box_to_qubit.index(diag1) + 1)
target2_bits = (box_to_qubit.index(diag2), box_to_qubit.index(diag2) + 1)

jump(src_bits, target1_bits, q_board)
# split(src_bits, target1_bits, target2_bits, q_board)
# merge(target1_bits, target2_bits, src_bits, q_board)
#qi.Statevector.from_instruction(q_board.reverse_bits()).to_dict()


q_board.measure([i for i in range(36)], [i for i in range(35, -1, -1)])
job = execute(q_board,Aer.get_backend('qasm_simulator'),shots=256)
counts = job.result().get_counts(q_board)
print(counts) # counts is a dictionary


NameError: name 'pos_dic' is not defined

## Simple Classical Captureture 

We need to know the diagonals upto to two levels of boxes and then we apply the following matrix:


In [ ]:
def updateColSplit(t1,t2,s,ancila):
     bell.cnot(s,ancila)
     bell.swap(s,t1)
     bell.swap(ancila,t2)

def updateCColSplit(to1,to2,t1,t2,s,ancila):
    # bell.cnot(s,ancila)
     bell.swap(ancila,s)
     bell.cswap(to1,ancila,t1)
     bell.cswap(to2,ancila,t2)

def updateColCaptureture(t,p,s,ancila):
     bell.swap(s,t)
     bell.cnot(p,ancila)
     bell.cnot(ancila,p)



## Quantum Split



In [ ]:
from qiskit import execute, Aer
qCapture = QuantumRegister(27, "box")
cCapture = ClassicalRegister(27)
bell = QuantumCircuit(qCapture, cCapture)
## Setting states




## Setting the center box to have a white piece
bell.x(0)
bell.x(1)
bell.x(2)
bell.x(3)
bell.x(4)



bell.x(8)
bell.x(9)
bell.x(10)
bell.x(11)
bell.x(12)

bell.x(8+13)
bell.x(9+13)
bell.x(10+13)
bell.x(11+13)
bell.x(12+13)

#bell.unitary(Capture,[0,8,4])
# bell.unitary(Capture,[1,9,5])



bell.unitary(Usplit,[9,7,6])
#updateColSplit(t1=6+13,t2=7+13,s=9+13,ancila=26)
updateCColSplit(to1=6,to2=7,t1=6+13,t2=7+13,s=9+13,ancila=26)




bell.measure(list(range(27)), list(range(26,-1,-1)))


job = execute(bell,Aer.get_backend('qasm_simulator'),shots=10)
counts = job.result().get_counts(bell)
print(counts) # counts is a dictionary
# bell.draw()

def print_state_board2(counts, size):
    state = []

    counts_temp = {}

    for i in counts:
        i_ = i.replace(" ", "")
        counts_temp[i_] = counts[i]

    counts = counts_temp
    
    count = 0
    count_alt = False
    row = ""

    #print(counts)

    for i in range(0, 13):

        # i = i//2

        box = " "
        
        for k in counts:
            if k[i] == '1':
                if k[i+13] == '1':
                    box = "B"
                else:
                    box = "W"
        
        row += box + '\t\t'

        count+=1

        if (count == 3 and not count_alt) or (count == 2 and count_alt):
            count = 0
            if count_alt:
                state.append("\t" + row[:-2])
            else:
                state.append(row[:-2])

            count_alt = not count_alt
            # print(row[0])
            row = ""


    state = state[::-1]

    for i in state[:-1]:
        print(i)
        print("-----------------" * (size//2))
    
    print(state[-1])


print_state_board2(counts, 5)

{'111110011011100000001101110': 7, '111110101011100000010101110': 3}
B		B		B
----------------------------------
	B		 
----------------------------------
 		B		B
----------------------------------
	W		W
----------------------------------
W		W		W


In [ ]:
# Captureture Classic

Capture=[[1,0,0,0,0,0,0,0],
     [0,0,0,0,1,0,0,0],
     [0,0,0,0,0,1,0,0],
     [0,0,0,0,0,0,1,0],
     [0,1,0,0,0,0,0,0],
     [0,0,1,0,0,0,0,0],
     [0,0,0,1,0,0,0,0],
     [0,0,0,0,0,0,0,1]]

Capturedag=np.matrix(Capture).getH()

## intializing the circuit
from qiskit import execute, Aer
qCapture = QuantumRegister(27, "box")
cCapture = ClassicalRegister(27)
bell = QuantumCircuit(qCapture, cCapture)

bell.x(0)
bell.x(1)
bell.x(2)
bell.x(3)
bell.x(4)


bell.x(8)
bell.x(9)
bell.x(10)
bell.x(11)
bell.x(12)


bell.x(8+13)
bell.x(9+13)
bell.x(10+13)
bell.x(11+13)
bell.x(12+13)

bell.unitary(Usplit,[9,7,6])
# #updateColSplit(t1=6+13,t2=7+13,s=9+13,ancila=26)
updateCColSplit(to1=6,to2=7,t1=6+13,t2=7+13,s=9+13,ancila=26)

# bell.unitary(Capture,[9,3,6])
# updateColCaptureture(s=9+13,t=3+13,p=6+13,ancila=26)

# bell.unitary(Capture,[0,6,3])
# updateColCaptureture(s=13,t=6+13,p=3+13,ancila=26)

# bell.swap(12,9)
# bell.swap(12+13,9+13)

# bell.unitary(Capture,[9,6,3])
# updateColCaptureture(s=9+13,t=3+13,p=6+13,ancila=26)

bell.measure(list(range(27)), list(range(26,-1,-1)))
job = execute(bell,Aer.get_backend('qasm_simulator'),shots=10)
counts = job.result().get_counts(bell)
# counts is a dictionary
# bell.draw()
print(counts)
def print_state_board2(counts, size):
    state = []

    counts_temp = {}

    for i in [list((dict(counts).keys()))[0]]:
        i_ = i.replace(" ", "")
        counts_temp[i_] = counts[i]

    counts = counts_temp
    
    count = 0
    count_alt = False
    row = ""

    #print(counts)

    for i in range(0, 13):

        # i = i//2

        box = " "
        
        for k in counts:
            if k[i] == '1':
                if k[i+13] == '1':
                    box = "B"
                else:
                    box = "W"
        
        row += box + '\t\t'

        count+=1

        if (count == 3 and not count_alt) or (count == 2 and count_alt):
            count = 0
            if count_alt:
                state.append("\t" + row[:-2])
            else:
                state.append(row[:-2])

            count_alt = not count_alt
            # print(row[0])
            row = ""


    state = state[::-1]

    for i in state[:-1]:
        print(i)
        print("-----------------" * (size//2))
    
    print(state[-1])


print_state_board2(counts, 5)
bell.draw()


{'111110011011100000001101110': 4, '111110101011100000010101110': 6}
B		B		B
----------------------------------
	B		 
----------------------------------
 		 		B
----------------------------------
	W		W
----------------------------------
W		W		W


┌───┐                                 ┌─┐                              »
 box_0: ┤ X ├─────────────────────────────────┤M├──────────────────────────────»
        ├───┤                                 └╥┘┌─┐                           »
 box_1: ┤ X ├──────────────────────────────────╫─┤M├───────────────────────────»
        ├───┤                                  ║ └╥┘┌─┐                        »
 box_2: ┤ X ├──────────────────────────────────╫──╫─┤M├────────────────────────»
        ├───┤                                  ║  ║ └╥┘┌─┐                     »
 box_3: ┤ X ├──────────────────────────────────╫──╫──╫─┤M├─────────────────────»
        ├───┤                                  ║  ║  ║ └╥┘┌─┐                  »
 box_4: ┤ X ├──────────────────────────────────╫──╫──╫──╫─┤M├──────────────────»
        └───┘┌─┐                               ║  ║  ║  ║ └╥┘                  »
 box_5: ─────┤M├───────────────────────────────╫──╫──╫──╫──╫───────────────────»
             └╥┘┌──────────┐                   ║  ║  ║  ║  ║                   »
 box_6: ──────╫─┤2         ├───────────────────╫──╫──╫──╫──╫───────────────────»
              ║ │          │                   ║  ║  ║  ║  ║                   »
 box_7: ──────╫─┤1         ├───────────────────╫──╫──╫──╫──╫───────────────────»
        ┌───┐ ║ │  Unitary │                   ║  ║  ║  ║  ║ ┌─┐               »
 box_8: ┤ X ├─╫─┤          ├───────────────────╫──╫──╫──╫──╫─┤M├───────────────»
        ├───┤ ║ │          │                   ║  ║  ║  ║  ║ └╥┘               »
 box_9: ┤ X ├─╫─┤0         ├───────────────────╫──╫──╫──╫──╫──╫────────────────»
        ├───┤ ║ └──────────┘                   ║  ║  ║  ║  ║  ║ ┌─┐            »
box_10: ┤ X ├─╫────────────────────────────────╫──╫──╫──╫──╫──╫─┤M├────────────»
        ├───┤ ║                                ║  ║  ║  ║  ║  ║ └╥┘┌─┐         »
box_11: ┤ X ├─╫────────────────────────────────╫──╫──╫──╫──╫──╫──╫─┤M├─────────»
        ├───┤ ║                                ║  ║  ║  ║  ║  ║  ║ └╥┘┌─┐      »
box_12: ┤ X ├─╫────────────────────────────────╫──╫──╫──╫──╫──╫──╫──╫─┤M├──────»
        └───┘ ║     ┌─┐                        ║  ║  ║  ║  ║  ║  ║  ║ └╥┘      »
box_13: ──────╫─────┤M├────────────────────────╫──╫──╫──╫──╫──╫──╫──╫──╫───────»
              ║     └╥┘     ┌─┐                ║  ║  ║  ║  ║  ║  ║  ║  ║       »
box_14: ──────╫──────╫──────┤M├────────────────╫──╫──╫──╫──╫──╫──╫──╫──╫───────»
              ║      ║      └╥┘┌─┐             ║  ║  ║  ║  ║  ║  ║  ║  ║       »
box_15: ──────╫──────╫───────╫─┤M├─────────────╫──╫──╫──╫──╫──╫──╫──╫──╫───────»
              ║      ║       ║ └╥┘┌─┐          ║  ║  ║  ║  ║  ║  ║  ║  ║       »
box_16: ──────╫──────╫───────╫──╫─┤M├──────────╫──╫──╫──╫──╫──╫──╫──╫──╫───────»
              ║      ║       ║  ║ └╥┘┌─┐       ║  ║  ║  ║  ║  ║  ║  ║  ║       »
box_17: ──────╫──────╫───────╫──╫──╫─┤M├───────╫──╫──╫──╫──╫──╫──╫──╫──╫───────»
              ║      ║       ║  ║  ║ └╥┘┌─┐    ║  ║  ║  ║  ║  ║  ║  ║  ║       »
box_18: ──────╫──────╫───────╫──╫──╫──╫─┤M├────╫──╫──╫──╫──╫──╫──╫──╫──╫───────»
              ║      ║       ║  ║  ║  ║ └╥┘    ║  ║  ║  ║  ║  ║  ║  ║  ║       »
box_19: ──────╫──────╫───────╫──╫──╫──╫──╫─────╫──╫──╫──╫──╫──╫──╫──╫──╫───────»
              ║      ║       ║  ║  ║  ║  ║     ║  ║  ║  ║  ║  ║  ║  ║  ║       »
box_20: ──────╫──────╫───────╫──╫──╫──╫──╫─────╫──╫──╫──╫──╫──╫──╫──╫──╫───────»
        ┌───┐ ║      ║       ║  ║  ║  ║  ║     ║  ║  ║  ║  ║  ║  ║  ║  ║ ┌─┐   »
box_21: ┤ X ├─╫──────╫───────╫──╫──╫──╫──╫─────╫──╫──╫──╫──╫──╫──╫──╫──╫─┤M├───»
        ├───┤ ║      ║       ║  ║  ║  ║  ║     ║  ║  ║  ║  ║  ║  ║  ║  ║ └╥┘   »
box_22: ┤ X ├─╫──────╫───────╫──╫──╫──╫──╫──X──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫────»
        ├───┤ ║      ║       ║  ║  ║  ║  ║  │  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║ ┌─┐»
box_23: ┤ X ├─╫──────╫───────╫──╫──╫──╫──╫──┼──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫─┤M├»
        ├───┤ ║      ║       ║  ║  ║  ║  ║  │  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║ └╥┘»
box_24: ┤ X ├─╫──────╫───────╫──╫──╫──╫

In [ ]:
from qiskit import execute, Aer
qCapture = QuantumRegister(2, "box")
cCapture = ClassicalRegister(2)
bell = QuantumCircuit(qCapture, cCapture)

bell.x(0)
bell.cnot(0,1)
bell.cnot(1,0)



bell.measure([0,1],[1,0])
job = execute(bell,Aer.get_backend('qasm_simulator'),shots=32)
counts = job.result().get_counts(bell)
print(counts)
bell.draw()

{'01': 32}


┌───┐     ┌───┐┌─┐   
box_0: ┤ X ├──■──┤ X ├┤M├───
       └───┘┌─┴─┐└─┬─┘└╥┘┌─┐
box_1: ─────┤ X ├──■───╫─┤M├
            └───┘      ║ └╥┘
 c3: 2/════════════════╩══╩═
                       1  0

# Initial State Circuit

In [ ]:
diag_dic_white = {0:[3], 1:[3,4], 2:[4,5], 3:[6,7], 4:[7,8], 5:[8], 6:[9], 7:[9,10], 8:[10,11], 9:[12,13], 10:[13, 14], 11:[14]}
diag_dic_black = {3:[0,1], 4:[1,2], 5:[2], 6:[3], 7:[3,4], 8:[4,5], 9:[6,7], 10:[7,8], 11:[8], 12:[9], 13:[9,10], 14:[10,11]}

def box_diag(box_no, color):
    if color:
        return get_diag_black[box_no]
    else:
        return get_diag_white[box_no] 

from qiskit import execute, Aer

qcap = QuantumRegister(39, "box")
ccap = ClassicalRegister(36)
bell = QuantumCircuit(qcap, ccap)
## Setting states

## Setting bottom left box to have black piece


## Setting the center box to have a white piece
bell.x(0)
bell.x(1)
bell.x(2)
bell.x(3)
bell.x(4)
bell.x(5)

bell.x(12)
bell.x(13)
bell.x(14)
bell.x(15)
bell.x(16)
bell.x(17)


bell.x(12+18)
bell.x(13+18)
bell.x(14+18)
bell.x(15+18)
bell.x(16+18)
bell.x(17+18)

#bell.unitary(Cap,[0,8,4])
# bell.unitary(Cap,[1,9,5])


# bell.unitary(Usplit,[9,7,6])
# updateColSplit(t1=6+13,t2=7+13,s=9+13,ancila=26)

counts = {'111111000000111111000000000000111111': 20}
bell.draw()


┌───┐
 box_0: ┤ X ├
        ├───┤
 box_1: ┤ X ├
        ├───┤
 box_2: ┤ X ├
        ├───┤
 box_3: ┤ X ├
        ├───┤
 box_4: ┤ X ├
        ├───┤
 box_5: ┤ X ├
        └───┘
 box_6: ─────
             
 box_7: ─────
             
 box_8: ─────
             
 box_9: ─────
             
box_10: ─────
             
box_11: ─────
        ┌───┐
box_12: ┤ X ├
        ├───┤
box_13: ┤ X ├
        ├───┤
box_14: ┤ X ├
        ├───┤
box_15: ┤ X ├
        ├───┤
box_16: ┤ X ├
        ├───┤
box_17: ┤ X ├
        └───┘
box_18: ─────
             
box_19: ─────
             
box_20: ─────
             
box_21: ─────
             
box_22: ─────
             
box_23: ─────
             
box_24: ─────
             
box_25: ─────
             
box_26: ─────
             
box_27: ─────
             
box_28: ─────
             
box_29: ─────
        ┌───┐
box_30: ┤ X ├
        ├───┤
box_31: ┤ X ├
        ├───┤
box_32: ┤ X ├
        ├───┤
box_33: ┤ X ├
        ├───┤
box_34: ┤ X ├
        ├───┤
box_35: ┤ X ├
        └───┘
box_36: ─────
             
box_37: ─────
             
box_38: ─────
             
 c0: 36/═════

Trial Code


In [ ]:
bell.measure(list(range(36)), list(range(35,-1,-1)))
job = execute(bell,Aer.get_backend('qasm_simulator'),shots=1)
counts = job.result().get_counts(bell)
print(counts)

# Fully Integrated Circuit + GUI

<!-- up_diag = {0:[-1, 3], 1:[3, 4], 2:[4, 5], 3:[6, 7], 4:[7, 8], 5:[8, -1], 6:[-1, 9], 7:[9, 10], 8:[10, 11], 9:[12, 13], 10:[13, 14], 11:[14, -1], 12:[-1, 15], 13:[15, 16], 14:[16, 17], 15:[-1, -1], 16:[-1, -1], 17:[-1, -1]}

down_diag = {0:[-1, -1], 1:[-1, -1], 2:[-1, -1], 3:[0, 1], 4:[1, 2], 5:[2, -1], 6:[-1, 3], 7:[3, 4], 8:[4, 5], 9:[6, 7], 10:[7, 8], 11:[8, -1], 12:[-1, 9], 13:[9, 10], 14:[10, 11], 15:[12, 13], 16:[13, 14], 17:[14, -1]}

def get_new_circuit(counts):
    qc = QuantumCircuit(qcap, ccap)
    
    count = 0

    for i in list(counts.keys())[0][0:36]:
        if i ==  '1':
            qc.x(count)
        
        count +=1
    
    return qc


def print_state_board2(counts, size):
    state = []

    counts_temp = {}

    for i in counts:
        i_ = i.replace(" ", "")
        counts_temp[i_] = counts[i]

    counts = counts_temp
    
    count = 0
    count_alt = False
    row = ""

    # print(counts),

    for i in range(0, 18):

        # i = i//2,

        box = " "
        
        for k in counts:
            if k[i] == '1':
                if k[i+18] == '1':
                    if box == " ":
                        box = "B"
                    else:
                        box += "B"
                else:
                    if box == " ":
                        box = "W"
                    else:
                        box += "W"
        
        # print(type(row), type(box))
        row += box + '\t\t'

        count+=1

        if count == 3:
            count = 0
            if count_alt:
                state.append("\t\t" + row[:-2])
            else:
                state.append(row[:-2])

            count_alt = not count_alt
            # print(row[0])
            row = ""


    state = state[::-1]

    output = ""
    
    for i in state[:-1]:
        # print(i)
        # print("-----------------" * (size//2))
        output += i + "\n"
        output += ("     " * (size//2)) + ("------------------" * (size//2)) + "\n"
    
    # print(state[-1])
    output += state[-1] + "\n"

    # print(output)
    
    return output


def update_classical_state(counts, s, t1, t2=None):

    print(s,t1,t2)

    for i in list(counts.keys())[0:1]:
        temp = list(i)

        temp[t1] = temp[s]
        temp[t1+18] = temp[s+18]

        if t2 != None:
            temp[t2] = temp[s]
            temp[t2+18] = temp[s+18]
        
        temp[s] = '0'
        temp[s+18] = '0'
        
        
        print(temp)
        
        temp = "".join(temp)
        
        counts[temp] = counts[i]
        del counts[i]
    
    # print_state_board2(counts, 5)
    print("\n\n\n")


from tkinter import *

root = Tk()  
root.geometry("400x490")  
root.title("test")

# canvas = Canvas(root)

V = StringVar()
label = Label(root, textvariable=V, bg = "white", bd=100, fg = "black")
label.pack()
# label.place(x=-160, y=0)


l1 = Label(root, text="source box: ")
l1.place(x=80, y=360)
s = Entry(root)
s.place(x=150,y=360)
# s.pack()

l2 = Label(root, text="t1 box: ")
l2.place(x=10, y=400)
t1 = Entry(root)
t1.place(x=50, y=400)
# t1.pack()

l3 = Label(root, text="t2 box: ")
l3.place(x=210, y=400)
t2 = Entry(root)
t2.place(x=250, y=400)
# t2.pack()


def gui_jump():
    jump_updated(int(t1.get()), int(s.get()), bell)
    # print(qi.Statevector.from_instruction(bell.reverse_bits()).to_dict())
    update_classical_state(counts, int(s.get()), int(t1.get()))

    V.set(print_state_board2(counts, 6))
    
def gui_sjump():
    split_jump_updated(int(t1.get()), int(t2.get()), int(s.get()), 36, bell)
    # print(qi.Statevector.from_instruction(bell.reverse_bits()).to_dict())
    update_classical_state(counts, int(s.get()), int(t1.get()), int(t2.get()))

    V.set(print_state_board2(counts, 6))

def gui_capture():
    if int(s.get()) < int(t1.get()):
        if (int(t1.get()) - int(s.get())) < 7:
            if up_diag[int(s.get())][0] != -1:
                path_qb = up_diag[int(s.get())][0]
                capture(int(t1.get()), int(s.get()), path_qb, 36, bell)
        else:
            if up_diag[int(s.get())][1] != -1:
                path_qb = up_diag[int(s.get())][1]
                capture(int(t1.get()), int(s.get()), path_qb, 36, bell)
                
        # capture(int(t1.get()), int(s.get()), int(s.get())+3, 36, bell)
    else:
        if (int(s.get()) - int(t1.get())) < 7:
            if down_diag[int(s.get())][1] != -1:
                path_qb = down_diag[int(s.get())][1]
                capture(int(t1.get()), int(s.get()), path_qb, 36, bell)
        else:
            if down_diag[int(s.get())][0] != -1:
                path_qb = down_diag[int(s.get())][0]
                capture(int(t1.get()), int(s.get()), path_qb, 36, bell)
        # capture(int(t1.get()), int(s.get()), int(s.get())-3, 36, bell)

def gui_measure():
    global bell
    bell.measure(list(range(36)), list(range(35,-1,-1)))
    # bell.measure([3,4,5,6,7], [4,3,2,1,0])
    job = execute(bell,Aer.get_backend('qasm_simulator'),shots=1)
    global counts
    counts = job.result().get_counts(bell)
    bell = get_new_circuit(counts)
    print(counts)
    V.set(print_state_board2(counts, 6))


b1 = Button(root, text ="Jump", command=gui_jump)
b1.place(x=120, y=425)

b2 = Button(root, text ="Split Jump", command=gui_sjump)
b2.place(x=170, y=425)
# b.pack()

b3 = Button(root, text ="Capture", command=gui_capture)
b3.place(x=250, y=425)

b4 = Button(root, text ="Measure", command=gui_measure)
b4.place(x=160, y=460)


V.set(print_state_board2(counts, 6))


root.mainloop() -->
